#### Importing libraries

In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import time
import pandas as pd

In [2]:

# Scraping method
def get_jobs(keyword, num_pages, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = Options()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)

    url = 'https://www.glassdoor.com/Job/us-' + keyword + '-jobs-SRCH_IL.0,2_IN1_KO3,17.htm?clickSource=searchBox'
    driver.get(url)

    company_name = []
    job_title = []
    salary_est = []
    location = []
    job_description = []
    salary_estimate = []
    company_size = []
    company_type = []
    company_sector = []
    company_industry = []
    company_founded = []
    company_revenue = []
    
    #Set current page to 1
    current_page = 1

    while current_page <= num_pages:  #If true, should be still looking for new jobs. 

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)
        
        
        
        #Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME, 'eigr9kq2')  #jl for Job Listing. These are the buttons we're going to click.
        print(len(job_buttons)) 
        
        for job_button in job_buttons:
            job_button.click()
            time.sleep(2)

            #Closes the signup prompt
            try:
                driver.find_element(By.XPATH, './/*[@id="JAModal"]/div/div[2]/span').click()  #clicking to the X.
            except NoSuchElementException:
                time.sleep(2)
                pass

            #Expands the Description section by clicking on Show More
            try:
                driver.find_element(By.XPATH, "//div[@class='css-t3xrds e856ufb4']").click()
                time.sleep(1)
            except NoSuchElementException:
                job_button.click()
                print(str(current_page) + '#ERROR: no such element')
                time.sleep(30)
                driver.find_element(By.XPATH, "//div[@class='css-t3xrds e856ufb4']").click()
            except ElementNotInteractableException:
                job_button.click()
                driver.implicitly_wait(30)
                print(str(current_page) + '#ERROR: not interactable')
                driver.find_element(By.XPATH, "//div[@class='css-t3xrds e856ufb4']").click()

            #Scrape 

            try:
                company_name.append(driver.find_element(By.XPATH, "//div[@class='css-87uc0g e1tk4kwz1']").text)
            except:
                company_name.append("#N/A")
                pass

            try:
                job_title.append(driver.find_element(By.XPATH, "//div[@class='css-1vg6q84 e1tk4kwz4']").text)
            except:
                job_title.append("#N/A")
                pass

            try:
                location.append(driver.find_element(By.XPATH, "//div[@class='css-56kyx5 e1tk4kwz5']").text)
            except:
                location.append("#N/A")
                pass

            try:
                job_description.append(driver.find_element(By.XPATH, "//div[@id='JobDescriptionContainer']").text)
            except:
                job_description.append("#N/A")
                pass

            try:
                salary_estimate.append(driver.find_element(By.XPATH, "//div[@class='css-1bluz6i e2u4hf13']").text)
            except:
                salary_estimate.append("#N/A")
                pass
            
            try:
                company_size.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Size']//following-sibling::*").text)
            except:
                company_size.append("#N/A")
                pass
            
            try:
                company_type.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Type']//following-sibling::*").text)
            except:
                company_type.append("#N/A")
                pass
                
            try:
                company_sector.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Sector']//following-sibling::*").text)
            except:
                company_sector.append("#N/A")
                pass
                
            try:
                company_industry.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Industry']//following-sibling::*").text)
            except:
                company_industry.append("#N/A")
                pass
                
            try:
                company_founded.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Founded']//following-sibling::*").text)
            except:
                company_founded.append("#N/A")
                pass
                
            try:
                company_revenue.append(driver.find_element(By.XPATH, "//div[@id='CompanyContainer']//span[text()='Revenue']//following-sibling::*").text)
            except:
                company_revenue.append("#N/A")
                pass
                    
                    
                    
                    
            done = True
                
       # Moves to the next page         
        if done:
            print(str(current_page) + ' ' + 'out of' +' '+ str(num_pages) + ' ' + 'pages done')
            driver.find_element(By.XPATH, "//span[@alt='next-icon']").click()   
            current_page = current_page + 1
            time.sleep(4)

    driver.close()
    df = pd.DataFrame({'company': company_name, 
    'job title': job_title,
    'location': location,
    'job description': job_description,
    'salary estimate': salary_estimate,
    'company_size': company_size,
    'company_type': company_type,
    'company_sector': company_sector,
    'company_industry' : company_industry, 'company_founded' : company_founded, 'company_revenue': company_revenue})

    df.to_csv("glassdoor_jobs.csv")
            

In [3]:
%%time
path = "./chromedriver"
get_jobs('data scientist', 10, path, 5)

C:\Users\Ray Joshi\AppData\Local\Temp\ipykernel_33520\3001969760.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


30
1 out of 10 pages done
30
2 out of 10 pages done
30
3#ERROR: no such element


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='css-t3xrds e856ufb4']"}
  (Session info: chrome=112.0.5615.86)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0074DCE3+50899]
	(No symbol) [0x006DE111]
	(No symbol) [0x005E5588]
	(No symbol) [0x006108F9]
	(No symbol) [0x00610AFB]
	(No symbol) [0x0063F902]
	(No symbol) [0x0062B944]
	(No symbol) [0x0063E01C]
	(No symbol) [0x0062B6F6]
	(No symbol) [0x00607708]
	(No symbol) [0x0060886D]
	GetHandleVerifier [0x009B3EAE+2566302]
	GetHandleVerifier [0x009E92B1+2784417]
	GetHandleVerifier [0x009E327C+2759788]
	GetHandleVerifier [0x007E5740+672048]
	(No symbol) [0x006E8872]
	(No symbol) [0x006E41C8]
	(No symbol) [0x006E42AB]
	(No symbol) [0x006D71B7]
	BaseThreadInitThunk [0x75657D49+25]
	RtlInitializeExceptionChain [0x7772B74B+107]
	RtlClearBits [0x7772B6CF+191]
